# Оценка эмоциональной окраски. Базовые методы

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pd.options.display.max_rows = 10

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Инициализация очистки и лемматизации

In [ ]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
from string import punctuation
russian_stopwords = stopwords.words("russian")

In [ ]:
!pip install -q gwpy

     |████████████████████████████████| 1.4 MB 4.8 MB/s 
     |████████████████████████████████| 9.4 MB 16.4 MB/s 
     |████████████████████████████████| 51 kB 5.7 MB/s 
     |████████████████████████████████| 45 kB 3.3 MB/s 
     |████████████████████████████████| 4.0 MB 36.0 MB/s 
     |████████████████████████████████| 295 kB 56.9 MB/s 
     |████████████████████████████████| 965 kB 57.0 MB/s 


In [ ]:
%%capture
!pip install natasha

from natasha import (
    Segmenter,
    MorphVocab,

    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,

    PER,

    Doc
)

In [ ]:
nat_segmenter = Segmenter()
nat_morph_vocab = MorphVocab()

nat_emb = NewsEmbedding()
nat_morph_tagger = NewsMorphTagger(nat_emb)
nat_syntax_parser = NewsSyntaxParser(nat_emb)
nat_ner_tagger = NewsNERTagger(nat_emb)

def lem_by_natasha_stopworded(text: str):
  text = text.lower()
  tokens = "".join([lit if lit not in punctuation else ' ' for lit in text]).split(" ")
  tokens = [token for token in tokens if len(token) > 0 and token not in russian_stopwords]
  text = " ".join(tokens)

  doc = Doc(text)
  doc.segment(nat_segmenter)
  doc.tag_morph(nat_morph_tagger)
  doc.parse_syntax(nat_syntax_parser)
  doc.tag_ner(nat_ner_tagger)

  new_text = ""
  for token in doc.tokens:
      token.lemmatize(nat_morph_vocab)
      new_text += token.lemma + " "

  return new_text

## Выгрузка и предобработка датасета

In [ ]:
feedback_sravnyator = pd.read_csv(f"drive/MyDrive/ОБЩЕЕ/РСОДПО/AI/emotional/data/forfinetuning/best_solyanka.csv").replace({'\n' : ' '}).dropna()
feedback_sravnyator

,text,labels
0,Основная проблематика курса включает:\n– Разви...,NEUTRAL
1,Курс «История книги и чтения» знакомит студент...,NEUTRAL
2,Изучение истории немецкой литературы будет про...,NEUTRAL
3,"Данный курс является частью minor \""Франция: и...",NEUTRAL
4,Курс «История французской литературы» построен...,NEUTRAL
...,...,...
4232,"Не надейтесь, что вы такой очень умный и талан...",NEGATIVE
4233,"Итак, жалею, что заканчиваю именно этот вуз. О...",NEGATIVE
4234,Содержание курсов не соответствуют заявленному...,NEGATIVE
4235,Мне так и не удалось поучиться в SkillFactory....,NEGATIVE


In [ ]:
feedback_sravnyator["stopworded_lem_text"] = feedback_sravnyator["text"].apply(lem_by_natasha_stopworded)
feedback_sravnyator.to_csv(f"drive/MyDrive/ОБЩЕЕ/РСОДПО/AI/emotional/data/best_solyanka_stopworded_natashed.csv")

## Выгрузка предобработанного датасета

In [ ]:
dataset = pd.read_csv('drive/MyDrive/ОБЩЕЕ/РСОДПО/AI/emotional/data/best_solyanka_stopworded_natashed.csv').dropna()
dataset

,Unnamed: 0,text,labels,stopworded_lem_text
0,0,Основная проблематика курса включает:\n– Разви...,NEUTRAL,основный проблематика курс включать n– развити...
1,1,Курс «История книги и чтения» знакомит студент...,NEUTRAL,курс « история книга чтение » знакомить студен...
2,2,Изучение истории немецкой литературы будет про...,NEUTRAL,изучение история немецкий литература проводить...
3,3,"Данный курс является частью minor \""Франция: и...",NEUTRAL,данный курс являться часть minor франция истор...
4,4,Курс «История французской литературы» построен...,NEUTRAL,курс « история французский литература » постро...
...,...,...,...,...
4232,4232,"Не надейтесь, что вы такой очень умный и талан...",NEGATIVE,надеяться очень умный талантливый точно смочь ...
4233,4233,"Итак, жалею, что заканчиваю именно этот вуз. О...",NEGATIVE,итак жалеть заканчивать именно вуз один назван...
4234,4234,Содержание курсов не соответствуют заявленному...,NEGATIVE,содержание курс соответствовать заявить лично ...
4235,4235,Мне так и не удалось поучиться в SkillFactory....,NEGATIVE,удаться поучиться skillfactory хотя туда очень...


In [ ]:
dataset["labels"] = dataset["labels"].replace({'NEUTRAL' : 0, 'POSITIVE' : 1, 'NEGATIVE' : -1})
dataset

,Unnamed: 0,text,labels,stopworded_lem_text
0,0,Основная проблематика курса включает:\n– Разви...,0,основный проблематика курс включать n– развити...
1,1,Курс «История книги и чтения» знакомит студент...,0,курс « история книга чтение » знакомить студен...
2,2,Изучение истории немецкой литературы будет про...,0,изучение история немецкий литература проводить...
3,3,"Данный курс является частью minor \""Франция: и...",0,данный курс являться часть minor франция истор...
4,4,Курс «История французской литературы» построен...,0,курс « история французский литература » постро...
...,...,...,...,...
4232,4232,"Не надейтесь, что вы такой очень умный и талан...",-1,надеяться очень умный талантливый точно смочь ...
4233,4233,"Итак, жалею, что заканчиваю именно этот вуз. О...",-1,итак жалеть заканчивать именно вуз один назван...
4234,4234,Содержание курсов не соответствуют заявленному...,-1,содержание курс соответствовать заявить лично ...
4235,4235,Мне так и не удалось поучиться в SkillFactory....,-1,удаться поучиться skillfactory хотя туда очень...


In [ ]:
dataset.labels.value_counts()

-1    1436
 1    1409
 0    1392
Name: labels, dtype: int64

## Выгрузка тестового датасета

In [ ]:
test_dataset = pd.read_csv('https://docs.google.com/spreadsheets/d/1kD4JRC_nDcF2ysmXKgYTA6jNpUi3vdbSmJY99lJxZis/export?format=csv')
test_dataset = test_dataset[["text", "y"]]
test_dataset

,text,y
0,Электив сам по себе скучноватый.,-1
1,"Я не считаю, что полученные знания помогут мне...",-1
2,Преподаватель очень тихо ведёт лекцию.,0
3,Преподаватель не пытается как-то ухудшить проц...,0
4,"Ничего не делаем, в конце 2 проекта, можно бра...",0
...,...,...
138,Такая теория никак не поможет ребятам заинтере...,-1
139,"Было бы гораздо лучше, будь преподаватель с ка...",-1
140,Или это преподавали бы люди с кафедры предприн...,0
141,"Толку учить пользоваться программой, где самый...",-1


In [ ]:
test_dataset.y.value_counts()

 0    59
 1    50
-1    34
Name: y, dtype: int64

## LR, SVM pipelines with TFIDF

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split

In [ ]:
X = dataset["stopworded_lem_text"].to_numpy()
y = dataset.labels.to_numpy()

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
X_train, X_test, y_train, y_test = X, test_dataset.text.apply(lem_by_natasha_stopworded).to_numpy(), y, test_dataset.y.to_numpy()

In [ ]:
logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(max_iter=10000)),
               ])

In [ ]:
%%time
logreg.fit(X_train, y_train)
precision_recall_fscore_support(y_test, logreg.predict(X_test), average='macro')

CPU times: user 1.65 s, sys: 2.24 s, total: 3.89 s
Wall time: 2.22 s


(0.5846091861402095, 0.5624592888002659, 0.5639249639249639, None)

In [ ]:
svm = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SVC(kernel="rbf",
                            class_weight="balanced",
                            C=10**3)),
               ])

In [ ]:
%%time
svm.fit(X_train, y_train)
precision_recall_fscore_support(y_test, svm.predict(X_test), average='macro')

CPU times: user 7.9 s, sys: 20.1 ms, total: 7.92 s
Wall time: 7.92 s


(0.5113954672778201, 0.5118311731472249, 0.50206009723158, None)

In [ ]:
linsvm = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SVC(kernel="linear",
                            class_weight="balanced",
                            C=10**3)),
               ])

In [ ]:
%%time
linsvm.fit(X_train, y_train)
precision_recall_fscore_support(y_test, linsvm.predict(X_test), average='macro')

CPU times: user 6.68 s, sys: 15 ms, total: 6.7 s
Wall time: 6.65 s


(0.4545694996822817, 0.45787304752409436, 0.4538552417539738, None)

## Вывод

Выдвинуто предположение, что базовые методы хороши, например, для классификации текстов по тематикам, поскольку учитываются вхождения тех или иных слов из разных предметных областей, но в классификации по окраске такой подход не подойдёт, поскольку одно и то же слово в зависимости от контекста может иметь разную окраску, разный смысл, поэтому необходимо обратить внимание на методы, учитывающие контекст, в частности – основанные на архитектуре Transformer.






#


In [6]:
import requests

text = "Обязанности:  монтаж, пуско-наладка охранных систем, охранно-пожарных систем, систем контроля доступа, систем видеонаблюдения," \
       "техническое сопровождение , выполнение пусконаладочных работ Требования: образование высшее техническое хорошее знание компьютера" \
       "знание технических характеристик элементов оборудования ОПС наличие знаний в области радиотехники и электротехники умение " \
       "программировать различные контрольные панели и системы ОПС (Андромеда, Болид, Sotel, Vista и т.д.) знание нормативных документов" \
       "по организации охраны объектов, пожарной безопасности Условия: официальное трудоустройство заработная плата – от 40000 рублей " \
       "возможны командировки, график 5/2 предоставляется необходимое оборудование для ремонтных диагностических, монтажных" \
       " и пусконаладочных работ"

In [7]:
%%time
requests.get(f"http://46.8.141.56:8080/?text={text}").json()

CPU times: user 16.2 ms, sys: 0 ns, total: 16.2 ms
Wall time: 329 ms


{'responsibilities': 'монтаж, пуско-наладка охранных систем, охранно-пожарных систем, систем контроля доступа, систем видеонаблюдения,техническое сопровождение , выполнение пусконаладочных работ',
 'requirements': 'образование высшее техническое хорошее знание компьютеразнание технических характеристик элементов оборудования ОПС наличие знаний в области радиотехники и электротехники умение программировать различные контрольные панели и системы ОПС (Андромеда, Болид, Sotel, Vista и т.д.) знание нормативных документовпо организации охраны объектов, пожарной безопасности',
 'terms': 'официальное трудоустройство заработная плата – от 40000 рублей возможны командировки, график 5/2 предоставляется необходимое оборудование для ремонтных диагностических, монтажных и пусконаладочных работ',
 'notes': ''}